<h1>
    <center>Dockless Vehicles Austin</center>
</h1>
<h2>
    <center>Team: Data Vaders</center>
    <h4>
        <center>Zarif Choudhury</center>
        <center>Vaidehi Durape</center>
        <center>Aashima Garg</center>
        <center>Aneesh Soni</center>
        <center>Brian Wilmarth</center>
    </h4>
</h2>

# Libraries Used

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import warnings
import folium
import cufflinks as cf
cf.go_offline()
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='aneeshsoni', api_key='e9PuzLMTHBbA5eVvZVNr')
warnings.filterwarnings('ignore')
mapbox_access_token = 'pk.eyJ1IjoiYW5lZXNoc29uaSIsImEiOiJjanZyOW1kemQwZnhwM3lueHhiZDE4ZTcwIn0.iywNpoytj-XU17BF4Tp_Fw'

import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
import IPython
from IPython.display import HTML, display, IFrame

%config InlineBackend.figure_format = 'png' #set 'png' here when working on notebook
%matplotlib inline

# Load Data

In [2]:
accidents = pd.read_csv("./Data/Scooter_Accidents.csv")

# Accident Data

### Summary of Data
- 165 total accident incidents
    - 9/5/18 to 11/29/18
- More males reported in accidents when compared to females
- Four main types of accidents
    1. Traffic Injury Pri 2
    2. Fall Pri 4
    3. Injury Pri 4
    4. Fall Pri 3
- Most accidents occurred during the afternoon and night (night and late night included)
- Most accidents occuring during the evening/night/late-night are concentrated downtown near 6th

### Formatting Data

In [3]:
accidents['Time Phone Pickup'] = pd.to_datetime(accidents['Time Phone Pickup'], format='%m/%d/%y %H:%M')

### Understanding Data

#### Dates

In [4]:
accidents.sort_values(by=['Time Phone Pickup'], inplace=True, ascending=False)
accidents.head(1)
accidents.tail(1)

,Contact Date,Day of Week,Longitude,Latitude,Time Phone Pickup,Incident Address,Incident Zip Code,Patient Gender,Problem Description,ePCR Destination Name
0,9/5/18,Wednesday,-97.829084,30.228721,2018-09-05 03:05:00,5700-6224 S MOPAC EXPY SB,78749,Female,Traffic Injury Pri 4F,South Austin Hospital


#### Accidents Split by Gender

In [16]:
accidents_male = accidents.loc[accidents['Patient Gender']=='Male']
accidents_female = accidents.loc[accidents['Patient Gender']=='Female']

labels = ['Males','Females']
values = [97,68]
colors = ['#37536d', '#1a76ff']

trace = go.Pie(labels=labels, values=values, title='Males vs Female Accidents',
               hoverinfo='label+percent', textinfo='value', 
               textfont=dict(size=20),
               marker=dict(colors=colors))

#py.iplot([trace], filename='styled_pie_chart')

fig = {
  "data": [
    {
      "values": [97,68],
      "labels": [
        "Males",
        "Females",
      ],
      'marker': {'colors': ['rgb(55, 83, 109)', 'rgb(26, 118, 255)']},
      "hoverinfo":"label+percent+name",
      "type": "pie",
    }],
  "layout": {
        "title":"Males vs Females Accidents"
    }
}

py.iplot(fig, filename='Males vs Females Accidents')

#### Accidents Split by Problem Description

In [7]:
accident_problems = pd.DataFrame(data = accidents['Problem Description'].value_counts())

accident_problems.columns = [
  'Count'
]

accident_problems['Accident_Type'] = accident_problems.index

trace1 = go.Bar(
    x = accident_problems['Accident_Type'],
    y = accident_problems['Count'],
    marker=dict(
        color='rgb(55, 83, 109)'
    )
)

data = [trace1]

layout = go.Layout(
    title='Accidents and their Occurrence',
    xaxis=dict(
        title='Type of Accident',
        titlefont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        ),
        tickfont=dict(
            size=8
        )
    ),
    yaxis=dict(
        title='Number of Calls',
        titlefont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
)

plotly.offline.plot(data, filename='./Visualizations/accident_problems.html', auto_open = False)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename = 'Accidents and their Occurrence')

#### Accidents Based on Time of Day

In [8]:
accidents = accidents.assign(Session = pd.cut(accidents['Time Phone Pickup'].dt.hour,[-1,5,11,17,20,24],labels=['Late Night', 'Morning', 'Afternoon', 'Evening', 'Night']))

accidents_male = accidents_male.assign(Session = pd.cut(accidents_male['Time Phone Pickup'].dt.hour,[-1,5,11,17,20,24],labels=['Late Night', 'Morning', 'Afternoon', 'Evening', 'Night']))
accidents_female = accidents_female.assign(Session = pd.cut(accidents_female['Time Phone Pickup'].dt.hour,[-1,5,11,17,20,24],labels=['Late Night', 'Morning', 'Afternoon', 'Evening', 'Night']))

In [9]:
accident_sessions_male = pd.DataFrame(data = accidents_male['Session'].value_counts())
accident_sessions_female = pd.DataFrame(data = accidents_female['Session'].value_counts())

accident_sessions_male.columns = [
  'Count',
]

accident_sessions_female.columns = [
  'Count',
]

accident_sessions_male['Session'] = accident_sessions_male.index
accident_sessions_female['Session'] = accident_sessions_female.index

trace1 = go.Bar(
    x = accident_sessions_male['Session'],
    y = accident_sessions_male['Count'],
    name='Males',
    marker=dict(
        color='rgb(55, 83, 109)'
    )
)

trace2 = go.Bar(
    x = accident_sessions_male['Session'],
    y = accident_sessions_female['Count'],
    name='Females',
    marker=dict(
        color='rgb(26, 118, 255)'
    )
)

data = [trace1, trace2]

layout = go.Layout(
    title='Accidents During Time of Day by Gender',
    xaxis=dict(
        title='Time of Day',
        titlefont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    yaxis=dict(
        title='Number of Accidents',
        titlefont=dict(
            size=14,
            color='rgb(107, 107, 107)'
        )
    ),
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)

plotly.offline.plot(data, filename='./Visualizations/accident_sessions.html', auto_open = False)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename = 'Accidents During Time of Day by Gender')

### All Accidents

- Green = Morning
- Red = Afternoon
- Light Blue = Evening
- Purple = Night
- Dark Blue = Late Night

In [29]:
Austin_Map = folium.Map(location = [30.267153, -97.7430608], zoom_start = 12)

for index, row in accidents.iterrows():
    description = 'DATE: ' + row['Contact Date'] + '\n' + 'GENDER: ' + row['Patient Gender'] + '\n' + 'PROBLEM: ' + row['Problem Description']
    if (row['Session'] == 'Morning'):
        folium.Marker([row['Latitude'], row['Longitude']], popup=folium.Popup(description, max_width = 110), icon=folium.Icon(color='green')).add_to(Austin_Map)
    elif (row['Session'] == 'Afternoon'):
        folium.Marker([row['Latitude'], row['Longitude']], popup=folium.Popup(description, max_width = 110), icon=folium.Icon(color='red')).add_to(Austin_Map)
    elif (row['Session'] == 'Night'):
        folium.Marker([row['Latitude'], row['Longitude']], popup=folium.Popup(description, max_width = 110), icon=folium.Icon(color='purple')).add_to(Austin_Map)
    elif (row['Session'] == 'Late Night'):
        folium.Marker([row['Latitude'], row['Longitude']], popup=folium.Popup(description, max_width = 110), icon=folium.Icon(color='darkblue')).add_to(Austin_Map)
    elif (row['Session'] == 'Evening'):
        folium.Marker([row['Latitude'], row['Longitude']], popup=folium.Popup(description, max_width = 110), icon=folium.Icon(color='lightblue')).add_to(Austin_Map)


Austin_Map.save("./Visualizations/Downtown_Austin.html")

In [34]:
%%HTML
<iframe width="100%" height="700" src="./Visualizations/Downtown_Austin.html"></iframe>